In [123]:
import numpy as np
import pandas as pd 
import os
import sys
import cv2 

import os
import datetime
import copy
import re
import yaml
import uuid
import warnings
import time
import inspect
import random

In [124]:
DATA_PATH = '/home/ubuntu/modeling/UCF_CRIME/data'
VIDEO_PATH = os.path.join(DATA_PATH, 'video')
ANNOTATION_PATH = os.path.join(DATA_PATH, 'annotation')

In [125]:
columns = ['video_name']

In [126]:
train_df = pd.read_csv(os.path.join(ANNOTATION_PATH, 'Train_Annotation.txt'), header=None, names=columns)
test_df = pd.read_csv(os.path.join(ANNOTATION_PATH, 'Test_Annotation.txt'), header=None, names=columns)

train_df

,video_name
0,Vandalism/Vandalism048_x264.mp4 7183
1,Training_Normal_Videos_Anomaly/Normal_Videos82...
2,Robbery/Robbery003_x264.mp4 984
3,Assault/Assault046_x264.mp4 952
4,Training_Normal_Videos_Anomaly/Normal_Videos57...
...,...
1605,Burglary/Burglary073_x264.mp4 2769
1606,Training_Normal_Videos_Anomaly/Normal_Videos05...
1607,Training_Normal_Videos_Anomaly/Normal_Videos24...
1608,Burglary/Burglary022_x264.mp4 2838


In [127]:
DOCKER_VIDEO_PATH = '/workspace/UCF_CRIME/data/video/'

In [128]:
len(train_df), len(test_df)

(1610, 290)

In [129]:
anomal_list = ['Arson', 'Assault', 'Stealing', 'Training_Normal_Videos_Anomaly', 'Testing_Normal_Videos_Anomaly']

new_tr_video = []
new_te_video = []

for data in train_df['video_name']:
    ab_type, video_name = data.split('/')
    new_video_name, new_video_frame = video_name.split()[0], video_name.split()[1]
#     print(new_video_frame)
    if ab_type in anomal_list: 
        new_tr_video.append(ab_type + '/' + new_video_name + ' ' + new_video_frame)

In [130]:
for data in test_df['video_name']:
    ab_type, video_name = data.split('/')
    new_video_name, new_video_frame = video_name.split()[0], video_name.split()[1]
#     print(new_video_frame)
    if ab_type in anomal_list: 
        new_te_video.append(ab_type + '/' + new_video_name + ' ' + new_video_frame)

len(new_tr_video), len(new_te_video)

(983, 167)

In [131]:
new_train_df = pd.DataFrame(new_tr_video, columns=['video_name'])
new_test_df = pd.DataFrame(new_te_video, columns=['video_name'])

In [132]:
def return_idx(ab_type):
    if ab_type == 'Arson':
        return 0
    elif ab_type == 'Assault':
        return 1 
    elif ab_type == 'Stealing':
        return 2 
    elif ab_type == 'Training_Normal_Videos_Anomaly' or ab_type == 'Testing_Normal_Videos_Anomaly':
        return 3 

In [133]:
train_list = [] 
test_list = []

In [134]:
for data in new_train_df['video_name']:
    video, total_frame = os.path.join(VIDEO_PATH, data.split()[0]), data.split()[1]
    # print(os.path.exists(video)) 
    frame_num = int(total_frame) - 32
    if os.path.exists(video):
        train_list.append(data.split()[0] + ' ' + str(frame_num))
    else:
        continue

In [135]:
for data in new_test_df['video_name']:
    video, total_frame = os.path.join(VIDEO_PATH, data.split()[0]), data.split()[1]
    # print(os.path.exists(video))   
    # print(total_frame, str(int(total_frame) + 16))
    frame_num = int(total_frame) - 32
    if os.path.exists(video):
        test_list.append(data.split()[0] + ' ' + str(frame_num))
    else:
        continue

In [136]:
train_list

['Training_Normal_Videos_Anomaly/Normal_Videos826_x264.mp4 1625',
 'Assault/Assault046_x264.mp4 920',
 'Training_Normal_Videos_Anomaly/Normal_Videos813_x264.mp4 2683',
 'Training_Normal_Videos_Anomaly/Normal_Videos354_x264.mp4 1763',
 'Training_Normal_Videos_Anomaly/Normal_Videos315_x264.mp4 1499',
 'Training_Normal_Videos_Anomaly/Normal_Videos803_x264.mp4 2694',
 'Training_Normal_Videos_Anomaly/Normal_Videos470_x264.mp4 1168',
 'Training_Normal_Videos_Anomaly/Normal_Videos253_x264.mp4 484',
 'Training_Normal_Videos_Anomaly/Normal_Videos082_x264.mp4 1166',
 'Training_Normal_Videos_Anomaly/Normal_Videos194_x264.mp4 4109',
 'Training_Normal_Videos_Anomaly/Normal_Videos124_x264.mp4 1362',
 'Training_Normal_Videos_Anomaly/Normal_Videos586_x264.mp4 1556',
 'Training_Normal_Videos_Anomaly/Normal_Videos225_x264.mp4 19909',
 'Training_Normal_Videos_Anomaly/Normal_Videos558_x264.mp4 1605',
 'Training_Normal_Videos_Anomaly/Normal_Videos468_x264.mp4 762',
 'Training_Normal_Videos_Anomaly/Normal_V

In [137]:
new_train_df = pd.DataFrame(train_list, columns=['video_name'])
new_test_df = pd.DataFrame(test_list, columns=['video_name'])

In [138]:
train_list = []
test_list = [] 

In [139]:
for data in new_train_df['video_name']:
    if int(data.split()[1]) <= 12000:
        train_list.append(data)
        
for data in new_test_df['video_name']:
    if int(data.split()[1]) <= 12000:
        test_list.append(data)

In [140]:
len(train_list), len(test_list)

(668, 160)

In [141]:
new_train_df = pd.DataFrame(train_list, columns=['video_name'])
new_test_df = pd.DataFrame(test_list, columns=['video_name'])

In [142]:
new_train_list = []
new_test_list = []

In [143]:
def make_train_list(video_name, total_frame, ab_type):
    global new_train_list
    
    ab_num = return_idx(ab_type.split('/')[0])
    for idx in range(0, total_frame, 16):
        path_video = DOCKER_VIDEO_PATH + ab_type
        val = path_video + '/' + video_name + ' ' + str(idx) + ' ' + str(ab_num)
        new_train_list.append(val)

In [144]:
new_train_df['video_name'][0].split('/')[0]

'Training_Normal_Videos_Anomaly'

In [145]:
for idx, data in enumerate(new_train_df['video_name']):
    ab_type = data.split('/')[0]
    video_name, total_frame = data.split('/')[1].split(' ')[0], data.split('/')[1].split(' ')[1]
    make_train_list(video_name, int(total_frame), ab_type)

In [146]:
def make_test_list(video_name, total_frame, ab_type):
    global new_test_list
    
    ab_num = return_idx(ab_type.split('/')[0])
    for idx in range(0, total_frame, 16):
        path_video = DOCKER_VIDEO_PATH + ab_type
        val = path_video + '/' + video_name + ' ' + str(idx) + ' ' + str(ab_num)
        # print(idx)
        new_test_list.append(val)

In [147]:
for idx, data in enumerate(new_test_df['video_name']):
    ab_type = data.split('/')[0]
    video_name, total_frame = data.split('/')[1].split(' ')[0], data.split('/')[1].split(' ')[1]
    make_test_list(video_name, int(total_frame), ab_type)

In [148]:
train_df = pd.DataFrame(new_train_list, columns=columns, index=None)
test_df = pd.DataFrame(new_test_list, columns=columns, index=None)

In [149]:
train_df['video_name'][0]

'/workspace/modeling/UCF_CRIME/data/videoTraining_Normal_Videos_Anomaly/Normal_Videos826_x264.mp4 0 3'

In [150]:
train_df.to_csv(os.path.join(ANNOTATION_PATH,'Anomaly_Train.lst'), header=0, index=None)
test_df.to_csv(os.path.join(ANNOTATION_PATH,'Anomaly_Test.lst'), header=0, index=None)

In [ ]:
path_video = DOCKER_VIDEO_PATH + data.split()[0]
train_list.append(path_video + ' ' + str(frame_num))